In [1]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import sys
import tensorflow as tf
import numpy as np
import imageio
import json
import random
import time
from run_nerf_helpers import *
from nerf_renderer import *
from misc_helpers import *
from load_llff import load_llff_data

tf.compat.v1.enable_eager_execution()

In [2]:
images, poses, bds, render_poses, i_test = load_llff_data('../data/rubiks_rgba/', 2,
                                                                  recenter=True, bd_factor=.75,
                                                                  spherify=True, rgba=True)

Loaded image data (500, 500, 4, 132) [ 500.         500.        1098.8775858]
Loaded ./data/rubiks_rgba/ 2.2962166918186226 11.288918670814095
Data:
(132, 3, 5) (132, 500, 500, 4) (132, 2)
HOLDOUT view is 128


In [3]:
i_test = np.arange(images.shape[0])[::8]
i_train = np.array([i for i in np.arange(int(images.shape[0])) if
                    (i not in i_test)])

In [4]:
hwf = poses[0, :3, -1]
poses = poses[:, :3, :4]

H, W, focal = hwf
H, W = int(H), int(W)
hwf = [H, W, focal]

print(H, W)

500 500


In [5]:
poses[0]

array([[-0.88162804, -0.39087975,  0.2644715 ,  0.23541795],
       [ 0.46857107, -0.7918595 ,  0.39166266,  0.35601556],
       [ 0.05633117,  0.46922433,  0.8812805 ,  0.8139291 ]],
      dtype=float32)

In [6]:
print('get rays')
rays = [get_rays_np(H, W, focal, p) for p in poses[:, :3, :4]]
rays = np.stack(rays, axis=0)  # [N, ro+rd, H, W, 3]
print('done, concats')
rays = np.transpose(rays, [0, 2, 3, 1, 4])
rays = np.reshape(rays, list(rays.shape[:3]) + [6])
rays_rgba = np.concatenate([rays, images], -1)
rays_rgba = np.stack([rays_rgba[i]
                        for i in i_train], axis=0)  # train images only
rays_rgba = np.reshape(rays_rgba, [-1, 10])
rays_rgba = rays_rgba.astype(np.float32)
print('shuffle rays')
np.random.shuffle(rays_rgba)
print('done')

get rays
done, concats
shuffle rays
done


In [9]:
i_batch = 0
N_rand = 1024
batch = rays_rgba[i_batch:i_batch+N_rand]  # [B, 2+1, 3*?]
batch[0]

array([ 0.95595187,  0.08639435,  0.28086206, -1.0113032 ,  0.12004511,
       -0.1679882 ,  0.        ,  0.        ,  0.        ,  0.        ],
      dtype=float32)

In [16]:
batch_rays = batch[:, :6]
target_rgb = batch[:, 6:9]
target_alpha = batch[:, 9]

batch_rays = np.reshape(batch_rays, [-1, 2, 3])
batch_rays = np.transpose(batch_rays, [1, 0, 2])
batch_rays.shape

(2, 1024, 3)

In [12]:
batch_rays[0, 0]

array([0.95595187, 0.08639435, 0.28086206], dtype=float32)

In [14]:
target_rgb.shape

(1024, 3)

In [18]:
target_alpha = np.reshape(target_alpha, [-1, 1])
target_alpha.shape

(1024, 1)

In [21]:
images[0][0, 0]

array([0., 0., 0., 0.], dtype=float32)